In [15]:
import os
import requests
import logging
import json
logging.basicConfig(level=logging.DEBUG)
session = requests.Session()
web_api='https://omero.matolab.org/api/'
#web_api='http://docker-dev.iwm.fraunhofer.de:5007/api/'
r = session.get(web_api)
#print(r)
base_url = r.json()["data"][-1]["url:base"]
all_urls = session.get(base_url).json()
logging.debug(all_urls)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): omero.matolab.org:443
DEBUG:urllib3.connectionpool:https://omero.matolab.org:443 "GET /api/ HTTP/1.1" 200 77
DEBUG:urllib3.connectionpool:https://omero.matolab.org:443 "GET /api/v0/ HTTP/1.1" 200 819
DEBUG:root:{'url:experimenters': 'https://omero.matolab.org/api/v0/m/experimenters/', 'url:experimentergroups': 'https://omero.matolab.org/api/v0/m/experimentergroups/', 'url:projects': 'https://omero.matolab.org/api/v0/m/projects/', 'url:datasets': 'https://omero.matolab.org/api/v0/m/datasets/', 'url:images': 'https://omero.matolab.org/api/v0/m/images/', 'url:screens': 'https://omero.matolab.org/api/v0/m/screens/', 'url:plates': 'https://omero.matolab.org/api/v0/m/plates/', 'url:rois': 'https://omero.matolab.org/api/v0/m/rois/', 'url:token': 'https://omero.matolab.org/api/v0/token/', 'url:servers': 'https://omero.matolab.org/api/v0/servers/', 'url:login': 'https://omero.matolab.org/api/v0/login/', 'url:save': 'https://omero.m

In [16]:
# get login session crsf token
token_url = all_urls["url:token"]
logging.info("getting session crsf token at {}".format(token_url))
try:
   token = session.get(token_url).json()["data"]
except Exception:
   logging.exception("Could not get Token!")
servers_url = all_urls["url:servers"]
# geting first server n list of servers
servers = session.get(servers_url).json()["data"]
#assume only one server is present
server = servers[0]

login_url = all_urls["url:login"]
credentials = {
            "username": os.getenv('OMERO_WEB_USERNAME',default='root'),
               "password": os.getenv('OMERO_WEB_PASSWORD', default=os.getenv('OMERO_ROOT_PASS', default='')),
               "csrfmiddlewaretoken": token,
               "server": server["id"]
            }

#login and test connection
logging.info("trying to login at {} with given credentials".format(login_url))
return_value = session.post(url=login_url,data=credentials,)
if return_value.status_code == 200:
    logging.info('login successful: {}'.format(return_value.json()))
else:
    logging.error('login failed: {}'.format(return_value))

INFO:root:getting session crsf token at https://omero.matolab.org/api/v0/token/
DEBUG:urllib3.connectionpool:https://omero.matolab.org:443 "GET /api/v0/token/ HTTP/1.1" 200 76
DEBUG:urllib3.connectionpool:https://omero.matolab.org:443 "GET /api/v0/servers/ HTTP/1.1" 200 77
INFO:root:trying to login at https://omero.matolab.org/api/v0/login/ with given credentials
DEBUG:urllib3.connectionpool:https://omero.matolab.org:443 "POST /api/v0/login/ HTTP/1.1" 200 284
INFO:root:login successful: {'success': True, 'eventContext': {'sessionId': 2400, 'sessionUuid': '5dce81a2-06aa-41b2-9581-8b8c6a243e1b', 'userId': 0, 'userName': 'root', 'groupId': 0, 'groupName': 'system', 'isAdmin': True, 'eventId': -1, 'eventType': 'User', 'memberOfGroups': [0, 1, 3], 'leaderOfGroups': [0]}}


In [17]:
# get image URL
images_url = all_urls["url:images"]
images_url
# get image data
images = session.get(images_url).json()
# get json data
#print(images)
for i in images["data"]:
            image_url = i["url:image"]
            print(image_url)
            json_data = session.get(image_url).json()
            json_data.update({"ImageUrl": image_url})
            #f = open(json_data["data"]["Name"]+".json", "w")
            f = open(os.path.join(os.getcwd(), json_data["data"]["Name"]+".json"), "w")
            f.write(json.dumps(json_data, indent=4, sort_keys=True))
            f.close()
            break

DEBUG:urllib3.connectionpool:https://omero.matolab.org:443 "GET /api/v0/m/images/ HTTP/1.1" 200 23053
DEBUG:urllib3.connectionpool:https://omero.matolab.org:443 "GET /api/v0/m/images/1/ HTTP/1.1" 200 9577


https://omero.matolab.org/api/v0/m/images/1/


In [10]:
# use blitz gateway
import os
import json
from omero.gateway import BlitzGateway
conn = BlitzGateway(
    os.getenv('OMERO_WEB_USERNAME',default='root'), 
    os.getenv('OMERO_WEB_PASSWORD', default=os.getenv('OMERO_ROOT_PASS', default='')),
    host="wss://wss.omero.matolab.org",secure=True)

conn.connect()

True